# Quick analysis

In [9]:
## Import libraries

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown
from utilities import redact_small_numbers


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# import data

df = pd.read_feather("../output/input.feather")
#df = pd.read_csv("../output/input.csv")

# for binary fields, replace zeros with null so that `count` can
binary_fields = ['elective_short_stay', 'elective_or_op']
for f in binary_fields:
    df[f] = df[f].astype('int64').replace(0,np.nan)

# treatment month
df['outpatient_covid_therapeutic_month'] = df['outpatient_covid_therapeutic_date'].astype(str).str[:-3]
df['inpatient_covid_therapeutic_month'] = df['inpatient_covid_therapeutic_date'].astype(str).str[:-3]

# latest date of inpatient records - to use as cutoff for all data
maxdate = df["any_admission_date"].max()
display(Markdown(f"Latest admission date: {maxdate}"))
display(Markdown(f"Therapeutics: {df['outpatient_covid_therapeutic_name'].unique().tolist()}"))

Latest admission date: 2022-04-25

Therapeutics: ['paxlovid', 'molnupiravir', 'sotrovimab', nan, 'remdesivir', 'casirivimab and imdevimab']

#### Summarise % MABs/AV treatments found in hospital records

In [12]:

fields = {
'Outpatient':
    ['outpatient_covid_therapeutic_date',
     'elective_or_op',
     'elective_short_stay', 
     'daycase_admission_date',
     'elective_x892_date', 
     'elective_x292_date', 
     'hospital_attendance_date'],
'Inpatient':      
    ['inpatient_covid_therapeutic_date',
     'any_admission_date', 
     'any_admission_x892_date',
     'any_admission_x292_date']
}

for x in fields:
    f = fields[x][0]
    display(Markdown(f"## {x}"))

    # filter to treatment dates within available SUS data range
    df1 = df.copy().loc[(df[f] <= maxdate)]

    if x=="Inpatient": # don't count admissions if discharge date was after treatment date
        display(Markdown(f"Note: for inpatients, recent spells may not yet have completed so some data may be missing"))
        for c in fields[x][1:]: # for each admission type
            # compare discharge date with treatment date and remove admission date if not in window
            df1.loc[df1[f] > df1[c.replace("admission", "discharge")], c] = np.nan
            
    # create df for mabs and separate df for Antivirals
    name_field = f'{x.lower()}_covid_therapeutic_name'
    df1[name_field] = df1[name_field].astype(str).fillna("")
    df_mab = df1.loc[df1[name_field].str.contains("mab")]
    df_non_mab = df1.loc[~df1[name_field].str.contains("mab")]
    treatments = {"MABs":df_mab,
                  "Antivirals":df_non_mab}

    # Breakdown by treatment type (MABs/Avs)
    for t in treatments:
        df_t = treatments[t]
        summary1 = pd.DataFrame(df_t[fields[x]].count()).rename(columns={0:"count"})
        summary1 = redact_small_numbers(summary1, n=5, rate_column=None)
        summary1["percent"] = (100*(summary1["count"]/summary1["count"][f])).fillna(0).round(1)
        summary1.index = summary1.index.str.replace("_date","").str.replace("_month","")
        out_text = f"""Out of **{summary1["count"][0]}** patients treated in *{x}* settings with *{t}*, **{summary1["count"][1]} ({summary1["percent"][1]}%)** had *{summary1.index[1].replace("_"," ")}* in SUS"""

        summary1.to_csv((f"../output/{x}_{t}.csv"))
        display(Markdown(f"### {t}"), Markdown(out_text), summary1)

    display(Markdown("### Breakdown by region, age group, month and treatment (MABs only)"))
    # Breakdown by region, age group and month (MABs only, most inclusive hospital record field only)
    for col in ["region_covid_therapeutics","age_group", f.replace("date","month"), 
                "outpatient_covid_therapeutic_name","inpatient_covid_therapeutic_name",
                "high_risk_cohort_covid_therapeutics"]:
        if (col=="outpatient_covid_therapeutic_name" and x=="Inpatient") or \
            (col=="inpatient_covid_therapeutic_name" and x=="Outpatient"):
            continue

        summary2 = pd.DataFrame(df_mab.groupby(col)\
                [fields[x][0:2]].count()).rename(columns={0:"count"})
        summary2 = redact_small_numbers(summary2, n=5, rate_column=None)
        summary2["percent"] = (100*summary2[fields[x][1]]/summary2[f]).round(1)
        # filter out zero/suppressed values
        summary2 = summary2.loc[summary2[f]>0]
        summary2.columns = summary2.columns.str.replace("_date","").str.replace("_month","")

        c_min = summary2["percent"].min()
        c_max = summary2["percent"].max()
        breakdown=col
        if col==f.replace("date","month"):
            breakdown = "treatment_month"
        out_text = f"""The percent of MABs patients treated in *{x}* settings found in SUS varied by *{breakdown}* from **{c_min}%** to **{c_max}%**"""
        summary2.to_csv((f"../output/{x}_{t}_by_{breakdown}.csv"))
        display( Markdown(out_text), summary2)

## Outpatient

### MABs

Out of **280** patients treated in *Outpatient* settings with *MABs*, **140 (50.0%)** had *elective or op* in SUS

,count,percent
outpatient_covid_therapeutic,280,100.0
elective_or_op,140,50.0
elective_short_stay,205,73.2
daycase_admission,150,53.6
elective_x892,30,10.7
elective_x292,25,8.9
hospital_attendance,35,12.5


### Antivirals

...

,inpatient_covid_therapeutic,any_admission,percent
inpatient_covid_therapeutic_month,,,
2021-12,40,25,62.5
2022-01,65,20,30.8
2022-02,65,20,30.8
2022-03,60,25,41.7
2022-04,55,10,18.2


The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *inpatient_covid_therapeutic_name* from **25.0%** to **36.8%**

,inpatient_covid_therapeutic,any_admission,percent
inpatient_covid_therapeutic_name,,,
casirivimab and imdevimab,40,10,25.0
sarilumab,70,25,35.7
sotrovimab,95,35,36.8
tocilizumab,80,25,31.2


The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *high_risk_cohort_covid_therapeutics* from **28.6%** to **66.7%**

,inpatient_covid_therapeutic,any_admission,percent
high_risk_cohort_covid_therapeutics,,,
Downs syndrome,30,10.0,33.3
HIV or AIDS,15,10.0,66.7
IMID,35,10.0,28.6
"IMID,solid cancer",35,NaN,NaN
"haematological diseases,stem cell transplant recipients",20,10.0,50.0
haematological malignancies,15,NaN,NaN
liver disease,10,NaN,NaN
primary immune deficiencies,30,10.0,33.3
"renal disease,sickle cell disease",25,10.0,40.0
